In [ ]:
import streamlit as st
import torch
import io
import numpy as np
import clip
import torchvision.transforms as T
from PIL import Image
from PIL import Image
import cv2
import difflib
from torchvision import transforms
from torchvision.models import swin_t
import torch.nn as nn
import torchvision.models as tvm
import torch
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, AutoModelForSeq2SeqLM
from langdetect import detect
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas

# ======================
# 🔧 Load Swin Transformer
# ======================
@st.cache_resource
def load_model():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = swin_t(weights=None)
    model.head = nn.Linear(model.head.in_features, 1)
    model.load_state_dict(torch.load("swin_lung_cancer.pth", map_location=device))
    model.to(device)
    model.eval()
    return model, device

model, device = load_model()

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# ======================
# 🌐 Load NLLB for Translation
# ======================
@st.cache_resource
def load_nllb():
    model_name = "facebook/nllb-200-distilled-600M"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
    return tokenizer, model

nllb_tokenizer, nllb_model = load_nllb()

def translate_nllb(text, source_lang, target_lang):
    inputs = nllb_tokenizer(text, return_tensors="pt")
    translated = nllb_model.generate(
        **inputs,
        forced_bos_token_id=nllb_tokenizer.convert_tokens_to_ids(target_lang),
        max_length=512
    )
    return nllb_tokenizer.decode(translated[0], skip_special_tokens=True)


# ======================
# 🧠 Load BioBERT for Q&A
# ======================
@st.cache_resource
def load_biobert():
    tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-base-cased-v1.1")
    model = AutoModelForQuestionAnswering.from_pretrained("dmis-lab/biobert-base-cased-v1.1")
    return tokenizer, model

qa_tokenizer, qa_model = load_biobert()

@st.cache_resource
def load_vit_classifier():
    from torchvision.models import vit_b_16
    vit = vit_b_16(pretrained=False)
    vit.heads.head = nn.Linear(vit.heads.head.in_features, 2)  # Assuming 2 classes: lung, non-lung
    vit.load_state_dict(torch.load("vit_lung_classifier.pth", map_location="cpu"))
    vit.eval()
    return vit

vit_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])  # Assuming grayscale normalization
])
vit_labels = ["lung", "non-lung"]
vit_model = load_vit_classifier()


@st.cache_resource
def load_clip_model():
    model, preprocess = clip.load("ViT-B/32", device="cpu")
    return model, preprocess

clip_model, clip_preprocess = load_clip_model()

# ======================
# 🌐 Language Options
# ======================
nllb_languages = {
    "English": "eng_Latn",
    "Hindi": "hin_Deva",
    "French": "fra_Latn",
    "Spanish": "spa_Latn",
    "Tamil": "tam_Taml",
    "Bengali": "ben_Beng",
    "Chinese (Simplified)": "zho_Hans",
    "Arabic": "arb_Arab",
    "German": "deu_Latn",
    "Japanese": "jpn_Jpan",
    "Korean": "kor_Hang",
    "Portuguese": "por_Latn",
    "Russian": "rus_Cyrl",
    "Italian": "ita_Latn",
    "Dutch": "nld_Latn",
    "Polish": "pol_Latn",
    "Turkish": "tur_Latn",
    "Vietnamese": "vie_Latn",
    "Thai": "tha_Thai",
    "Swahili": "swa_Latn",
    "Greek": "ell_Grek",
    "Ukrainian": "ukr_Cyrl",
    "Hebrew": "heb_Hebr",
    "Romanian": "ron_Latn",
    "Czech": "ces_Latn",
    "Hungarian": "hun_Latn",
    "Swedish": "swe_Latn",
    "Finnish": "fin_Latn",
    "Norwegian": "nor_Latn",
    "Danish": "dan_Latn",
    "Malay": "msa_Latn",
    "Indonesian": "ind_Latn",
    "Filipino": "fil_Latn",
    "Tamil (Sri Lanka)": "tam_Sinh",
    "Marathi": "mar_Deva",
    "Telugu": "tel_Telu",
    "Kannada": "kan_Kann",
    "Gujarati": "guj_Gujr",
    "Punjabi": "pan_Guru",
    "Nepali": "nep_Deva",
    "Sinhala": "sin_Sinh",
    "Khmer": "khm_Khmr",
    "Laotian": "lao_Laoo",
    "Mongolian": "mon_Cyrl",
    "Pashto": "pus_Afgh",
    "Farsi (Persian)": "fas_Pers",
    "Tigrinya": "tir_Tigr",
    "Somali": "som_Latn",
    "Amharic": "amh_Ethi",
    "Zulu": "zul_Latn",
    "Xhosa": "xho_Latn",
    "Quechua": "que_Latn",
    "Aymara": "aym_Latn",
    "Wolof": "wol_Latn",
    "Yoruba": "yor_Latn",
    "Igbo": "ibo_Latn",
    "Haitian Creole": "hat_Latn",
    "Basque": "eus_Latn",
    "Catalan": "cat_Latn",
    "Galician": "glg_Latn",
    "Serbian": "srp_Cyrl",
    "Croatian": "hrv_Latn",
    "Bosnian": "bos_Latn",
    "Albanian": "sqi_Latn",
    "Bulgarian": "bul_Cyrl",
    "Macedonian": "mkd_Cyrl",
    "Slovak": "slk_Latn",
    "Slovenian": "slv_Latn",
    "Estonian": "est_Latn",
    "Latvian": "lav_Latn",
    "Lithuanian": "lit_Latn",
    "Georgian": "geo_Geor",
    "Armenian": "hye_Latn",
    "Icelandic": "isl_Latn",
    "Welsh": "cym_Latn",
    "Irish": "gle_Latn",
    "Scottish Gaelic": "gla_Latn",
    "Finnish": "fin_Latn"
}
# ======================
# 🔧 UI Text Elements in English
# ======================
ui_texts = {
    "title": "🩻 Lung Cancer Detection & Multilingual Chatbot",
    "language_select": "🌐 Select your preferred language",
    "upload_ct_scan": "📤 Upload a CT scan image",
    "medical_report": "📝 Your Medical Report",
    "ask_question": "🤖 Ask a question about your report or general health",
    "submit_question": "💬 Your question",
    "export_pdf": "📥 Export Report as PDF",
    "telehealth_link": "💬 For further consultation, you can also talk to a doctor via telehealth: [Click here to consult](https://doctorondemand.com/)"
}

# ======================
# Translate UI Dynamically
# ======================
def translate_ui(text_dict, target_language):
    translated_texts = {}
    for key, text in text_dict.items():
        translated_texts[key] = translate_nllb(text, "eng_Latn", nllb_languages.get(target_language, "eng_Latn"))
    return translated_texts

# ======================
# FAQ Answers (Rule-based)
# ======================
def get_faq_answer(question):
    # Example FAQ answers
    question = question.lower()

    if "symptoms" in question:
        return "Common symptoms of lung cancer include persistent cough, chest pain, shortness of breath, coughing up blood, and fatigue."
    elif "treatment" in question:
        return "Treatment options for lung cancer may include surgery, radiation therapy, chemotherapy, targeted therapy, or immunotherapy, depending on the type and stage of cancer."
    elif "diagnosed" in question:
        return "Lung cancer is diagnosed through imaging tests like CT scans, biopsies, and PET scans."
    elif "reduce risk" in question:
        return "To reduce your risk of lung cancer, avoid smoking, avoid secondhand smoke, and maintain a healthy diet and lifestyle."
    else:
        return "Sorry, I don't have an answer for that. Please consult a healthcare professional."
# ======================
# 🖼️ Streamlit Interface
# ======================

# Language Selection with auto-detection fallback
tgt_language = st.selectbox("🌐 Select your preferred language", list(nllb_languages.keys()))
tgt_lang_code = nllb_languages[tgt_language]

# Translate UI texts to selected language
translated_ui = translate_ui(ui_texts, tgt_language)

# Display translated UI
st.title(translated_ui["title"])
st.markdown(f"📁 **{translate_nllb('Upload your CT scan image:', 'eng_Latn', tgt_lang_code)}**")
st.markdown(translate_nllb("Drag and drop file here", "eng_Latn", tgt_lang_code))
st.markdown(translate_nllb("Limit 200MB per file • JPG, JPEG, PNG", "eng_Latn", tgt_lang_code))
uploaded_file = st.file_uploader(translated_ui["upload_ct_scan"], type=["jpg", "jpeg", "png"])
def is_non_medical_image(img, model, preprocess):
    classes_to_flag = ["dog", "cat", "person", "photo", "illustration", "drawing", "painting", "graph", "chart", "diagram", "scene", "landscape"]
    image_input = preprocess(img).unsqueeze(0)
    with torch.no_grad():
        text_inputs = clip.tokenize(classes_to_flag)
        image_features = model.encode_image(image_input)
        text_features = model.encode_text(text_inputs)
        image_features /= image_features.norm(dim=-1, keepdim=True)
        text_features /= text_features.norm(dim=-1, keepdim=True)
        similarities = (100.0 * image_features @ text_features.T).squeeze(0)
        top_idx = torch.argmax(similarities).item()
    return classes_to_flag[top_idx]
def validate_ct_image(img: Image.Image, vit_model, vit_transform, labels, tgt_lang_code, translate):
    try:
        grayscale = img.convert("L")
        img_array = np.array(grayscale)
        height, width = img_array.shape

        print(f"✅ Shape: {img_array.shape}")

        if height < 150 or width < 150:
            return translate("This image is too small to be a CT scan. Please upload a larger image.", "eng_Latn", tgt_lang_code)

        mean_pixel = np.mean(img_array)
        std_pixel = np.std(img_array)
        print(f"✅ Mean Pixel: {mean_pixel} Std Dev: {std_pixel}")

        edges = cv2.Canny(img_array, 50, 150)
        edge_density = np.sum(edges > 0) / edges.size
        print(f"✅ Edge density: {edge_density}")

        if edge_density < 0.01 or edge_density > 0.2:
            return translate("This image has either too little or too much detail to be a CT scan.", "eng_Latn", tgt_lang_code)

        input_tensor = vit_transform(img.convert("RGB")).unsqueeze(0)
        print(f"✅ Tensor shape: {input_tensor.shape}")

        with torch.no_grad():
            output = vit_model(input_tensor)
            pred_idx = torch.argmax(output, dim=1).item()
            pred_label = labels[pred_idx]
            confidence = torch.nn.functional.softmax(output, dim=1)[0][pred_idx].item()

        print(f"🔍 ViT Prediction: {pred_label} (Confidence: {confidence:.2f})")

        if pred_label.lower() != "lung":
          return translate(f"This appears to be a scan of {pred_label}, not the lungs. Please upload a lung CT scan.", "eng_Latn", tgt_lang_code)
        return "Valid lung CT scan"
    except Exception as e:
        print("❌ Exception during validation:", str(e))
        return translate("There was a problem processing your scan. Please upload a standard CT scan image.", "eng_Latn", tgt_lang_code)

def final_ct_validation(img, vit_model, vit_transform, vit_labels, tgt_lang_code):
    message = validate_ct_image(img, vit_model, vit_transform, vit_labels, tgt_lang_code, translate_nllb)
    return message

if uploaded_file:
    img = Image.open(uploaded_file).convert("RGB")

    # Run CLIP filter first
    clip_label = is_non_medical_image(img, clip_model, clip_preprocess)
    if clip_label in ["dog", "cat", "person", "illustration", "drawing", "photo", "graph", "chart", "landscape"]:
        st.error(translate_nllb("❌ This image looks like a " + clip_label + " and is not a valid medical CT scan. Please upload a proper lung CT scan.", "eng_Latn", tgt_lang_code))
        st.stop()

    msg = final_ct_validation(img, vit_model, vit_transform, vit_labels, tgt_lang_code)
    if "Valid lung CT scan" not in msg:
        st.error("❌ " + msg)
        st.stop()

    # Step 1: Show image first
    st.image(img, caption=translate_nllb("✅ Valid lung CT scan", "eng_Latn", tgt_lang_code), use_container_width=True)

    # Step 2: Confirm it's a valid scan
    st.success(translate_nllb("✅ Valid lung CT scan", "eng_Latn", tgt_lang_code))

    # Step 3: Confirm report generation
    st.info(translate_nllb("✅ Report generated successfully!", "eng_Latn", tgt_lang_code))

    try:
        input_tensor = transform(img).unsqueeze(0).to(device)
    except Exception as e:
        st.error("⚠️ Unable to process the image. Please ensure it's a valid CT scan in PNG/JPEG format.")
        st.stop()

    with torch.no_grad():
        prediction = model(input_tensor).squeeze().item()

    def classify(score):
        if score < 1:
            return "Low"
        elif 1 <= score < 3:
            return "Moderate"
        else:
            return "High"

    category = classify(prediction)


    # Medical Report
    translated_report = f"""
🩺 **{translate_nllb('Your Medical Report', 'eng_Latn', tgt_lang_code)}**

✅ **{translate_nllb('What Did We See in the Scan?', 'eng_Latn', tgt_lang_code)}**
{translate_nllb('The scan reveals features that may be consistent with potentially malignant nodules. These include irregular shapes, larger size, or increased density in certain lung regions. While this is not a definitive diagnosis, it does warrant further clinical evaluation. A follow-up with your physician is strongly recommended. Based on the characteristics of the nodules observed, we used our AI system to calculate a malignancy risk score to assist in evaluating the likelihood of lung cancer.' if category == "High"
            else 'The scan shows some features that could be associated with early or borderline lung nodules. These may include small opacities, mildly irregular borders, or subtle density changes. While not immediately alarming, these findings should be monitored over time. Using these visual features, our AI system calculated a malignancy risk score to help estimate the potential for future complications, allowing for early intervention if necessary.' if category == "Moderate"
            else 'The scan does not reveal any significant abnormalities in the lung tissue. No suspicious nodules or irregular patterns were detected, and the overall structure appears normal. Even in the absence of concerning features, our AI model still computed a malignancy risk score to provide a baseline for your lung health and support continued screening over time. Great job taking care of your health!', 'eng_Latn', tgt_lang_code)}

📊 **{translate_nllb('Malignancy Risk Score', 'eng_Latn', tgt_lang_code)}**
{translate_nllb('Your AI-predicted risk score is', 'eng_Latn', tgt_lang_code)} **{round(prediction, 2)}**.
{translate_nllb('This puts your risk in the', 'eng_Latn', tgt_lang_code)} **{translate_nllb(category, 'eng_Latn', tgt_lang_code)}** {translate_nllb('category.', 'eng_Latn', tgt_lang_code)}

📌 **{translate_nllb('What is Malignancy Score?', 'eng_Latn', tgt_lang_code)}**
{translate_nllb('This is a number predicted by our AI model based on your lung scan. It indicates the likelihood of cancer:', 'eng_Latn', tgt_lang_code)}
- **{translate_nllb('Low (0 to 1):', 'eng_Latn', tgt_lang_code)}** {translate_nllb('Minimal signs of cancer', 'eng_Latn', tgt_lang_code)}
- **{translate_nllb('Moderate (1 to 3):', 'eng_Latn', tgt_lang_code)}** {translate_nllb('Some warning signs', 'eng_Latn', tgt_lang_code)}
- **{translate_nllb('High (3+):', 'eng_Latn', tgt_lang_code)}** {translate_nllb('Strong suspicion, further tests recommended', 'eng_Latn', tgt_lang_code)}

🩺 **{translate_nllb('What Should You Do Now?', 'eng_Latn', tgt_lang_code)}**
{translate_nllb('Based on the scan findings and your AI-generated malignancy risk score, we strongly recommend that you schedule a consultation with your physician as soon as possible. Further diagnostic tests, such as a PET scan, biopsy, or additional imaging, may be needed to assess the nature of the nodules.' if category == "High"
            else 'While the findings are not urgent, it is advisable to follow up with your healthcare provider for a routine evaluation. Periodic imaging and health monitoring can ensure that any changes in the nodules are detected early and managed appropriately.' if category == "Moderate"
            else 'No immediate action is necessary. However, staying proactive with regular health checkups and scheduled lung screenings is a good practice. Continue maintaining your lung health and consult your doctor if any new symptoms arise.', 'eng_Latn', tgt_lang_code)}
"""

    st.markdown(translated_report)


    # ====================
    # Expandable Sections
    # ====================
    with st.expander("🔍 " + translate_nllb("Learn More: How was this score calculated?", "eng_Latn", tgt_lang_code)):
      st.markdown(translate_nllb(
        "The malignancy score is based on an AI model trained using a large dataset of lung CT scans. It considers patterns in the scan that are often associated with cancerous growths. The model then calculates a risk score based on these patterns.",
        "eng_Latn", tgt_lang_code))

    with st.expander("⚠️ " + translate_nllb("Learn More: What are the limitations of this prediction?", "eng_Latn", tgt_lang_code)):
      st.markdown(translate_nllb(
        "The malignancy score is an AI-generated estimate and is not a definitive diagnosis. It can sometimes produce false positives or negatives. Always consult with a medical professional for a comprehensive evaluation.",
        "eng_Latn", tgt_lang_code))

    # ====================
    # Visual Score Indicator
    # ====================
    if category == "High":
        st.markdown(f'<div style="background-color:red;color:white;padding:5px;">{translate_nllb("High Risk", "eng_Latn", tgt_lang_code)} (Score: {round(prediction, 2)})</div>', unsafe_allow_html=True)
        st.markdown(translate_nllb("**💙 Emotional Support:** You are not alone. Support is available, and there are many treatment options for cancer. Stay strong!", "eng_Latn", tgt_lang_code))
    elif category == "Moderate":
        st.markdown(f'<div style="background-color:yellow;color:black;padding:5px;">{translate_nllb("Moderate Risk", "eng_Latn", tgt_lang_code)} (Score: {round(prediction, 2)})</div>', unsafe_allow_html=True)
    else:
        st.markdown(f'<div style="background-color:green;color:white;padding:5px;">{translate_nllb("Low Risk", "eng_Latn", tgt_lang_code)} (Score: {round(prediction, 2)})</div>', unsafe_allow_html=True)

    # ======================
    # ======================
st.subheader(translate_nllb("🤖 Ask a question about your report or general health", "eng_Latn", tgt_lang_code))
question = st.text_input(translate_nllb("💬 Your question", "eng_Latn", tgt_lang_code))

if question:
    try:
        lang_detected = detect(question)
    except:
        lang_detected = tgt_lang_code

    if lang_detected != tgt_lang_code:
        pass

    # Translate question to English for processing
    question_translated = translate_nllb(question, tgt_lang_code, "eng_Latn")

    # Check if the question matches an FAQ
    def get_faq_answer(question):
        question = question.lower().strip()
        faq_dict = {
            "what is lung cancer?": "Lung cancer is a type of cancer that begins in the lungs. It is one of the leading causes of cancer-related deaths.",
            "What is a lung scan?": "A lung scan is a picture of your lungs that helps us check if everything looks normal.",
            "What are nodules?": "Nodules are small spots or lumps that can appear in the lungs. Most of the time, they are not dangerous.",
            "What is the malignancy score?": "This is a number we use to guess how likely a spot in your lung might be serious. A higher number means more risk.",
            "How does the AI work?": "The computer looks at your scan and finds patterns that might mean something is wrong. It gives you a score and a risk level.",
            "Does a high score mean I have cancer?": "No. A high score only means you might need more tests to be sure. Only a doctor can tell for sure.",
            "What should I do if my score is high?": "It’s best to talk to a doctor. They might suggest more tests to learn more about your lungs.",
            "Can I trust this report?": "The report uses smart technology, but it's not a final answer. Always follow up with a real doctor.",
            "What if my score is low?": "That’s good news! But it’s still smart to get regular checkups and stay healthy.",
            "How often should I do a lung scan?": "Your doctor can tell you what’s best, especially if you smoke or have had health problems before.",
            "Why do people get lung problems?": "Some reasons include smoking, pollution, or being around harmful chemicals. Sometimes it just happens without a clear reason.",
            "I don’t feel sick—why does the scan show something?": "Lung problems can start without any signs. That’s why scans are helpful—they show things early.",
            "What if I don’t have a doctor?": "Try to visit a health center or clinic nearby. They can help you understand your scan and what to do next.",
            "Can I show this report to a doctor?": "Yes, please do! This report is made to help your doctor understand your scan better.",
            "Will I need more tests?": "Maybe. The scan gives a clue, but doctors might ask for more tests to be sure everything is okay.",
            "How long does it take to find out the full result?": "If you go to a doctor, they may give you answers quickly or ask for more time if more tests are needed.",
            "Can young people have lung problems too?": "Yes, but it’s less common. Lung checks are important for everyone, especially if they have a cough, chest pain, or smoke.",
            "How can I keep my lungs healthy?": "Don’t smoke, breathe clean air, eat healthy food, and go for regular health checkups.",
            "Is it okay to feel scared about this?": "Yes, it’s normal. But don’t worry alone—talk to someone you trust or visit a doctor.",
            "Can I use this tool again in the future?": "Yes! You can use it again any time you have a new scan.",
            "what are the symptoms of lung cancer?": "Common symptoms include coughing, chest pain, shortness of breath, and weight loss.",
            "what is the treatment for lung cancer?": "Treatment options may include surgery, chemotherapy, radiation therapy, immunotherapy, or targeted therapy.",

        }
        best_match = difflib.get_close_matches(question, faq_dict.keys(), n=1, cutoff=0.7)
        if best_match:
          best_match = difflib.get_close_matches(question, faq_dict.keys(), n=1, cutoff=0.6)
          if best_match:
            return faq_dict[best_match[0]]
        return None

    faq_answer = get_faq_answer(question_translated)

    if faq_answer:
        translated_answer = translate_nllb(faq_answer, "eng_Latn", tgt_lang_code)
        st.markdown(f"🩺 **{translate_nllb('Answer', 'eng_Latn', tgt_lang_code)}:** {translated_answer}")
    else:
        # Use BioBERT for custom question answering
        def get_biobert_answer(question, context):
            inputs = qa_tokenizer(question, context, return_tensors="pt")
            outputs = qa_model(**inputs)
            answer_start = torch.argmax(outputs.start_logits)
            answer_end = torch.argmax(outputs.end_logits)
            answer = qa_tokenizer.convert_tokens_to_string(
                qa_tokenizer.convert_ids_to_tokens(inputs.input_ids[0][answer_start:answer_end + 1])
            )
            return answer

        medical_report_context = f"""{translated_report}"""
        answer = get_biobert_answer(question_translated, medical_report_context).strip()

        # Fallback if BioBERT fails
        if not answer or "[UNK]" in answer or len(answer) < 3:
          answer = "Sorry, I couldn't find a clear answer. Please consult a doctor for guidance."

        answer_translated = translate_nllb(answer, "eng_Latn", tgt_lang_code)
        st.markdown(f"🩺 **{translate_nllb('Answer', 'eng_Latn', tgt_lang_code)}:** {answer_translated}")

    # ============================
    # Telehealth Link
    # ============================
    telehealth_link = "https://doctorondemand.com/"
    st.markdown(f"💬 **{translate_nllb('For further consultation, you can also talk to a doctor via telehealth', 'eng_Latn', tgt_lang_code)}:** [**{translate_nllb('Click here to consult', 'eng_Latn', tgt_lang_code)}**]({telehealth_link})")

    # ============================
    # ============================
# Export Report as PDF
# ============================
def export_pdf(report_text):
    from reportlab.lib.pagesizes import letter
    from reportlab.pdfgen import canvas
    import textwrap

    buffer = io.BytesIO()
    c = canvas.Canvas(buffer, pagesize=letter)

    # Set font and position
    c.setFont("Helvetica", 11)
    x, y = 40, 750
    line_height = 15

    # Wrap text at 100 characters per line
    wrapped_lines = []
    for paragraph in report_text.split("\n"):
        wrapped_lines.extend(textwrap.wrap(paragraph, width=100))
        wrapped_lines.append("")  # Add empty line between paragraphs

    for line in wrapped_lines:
        if y < 50:
            c.showPage()
            y = 750
            c.setFont("Helvetica", 11)
        c.drawString(x, y, line)
        y -= line_height

    c.showPage()
    c.save()
    buffer.seek(0)
    return buffer

if st.button(translated_ui["export_pdf"]):
    pdf_buffer = export_pdf(translated_report)
    st.download_button("Download PDF", pdf_buffer, file_name="lung_cancer_report.pdf", mime="application/pdf")


ModuleNotFoundError: No module named 'streamlit'